In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
def add_gaussian_noise(data, noise_factor=0.1):
    std = torch.std(data, dim=0, keepdim=True)  # 计算原始数据的标准差
    noise = torch.randn_like(data) * (std * noise_factor)  # 生成噪声
    return data + noise
def add_positional_encoding(x):
    """
    x: Tensor of shape (batch_size, seq_len, d_model)
    """
    batch_size, seq_len, d_model = x.shape

    # 初始化位置编码矩阵 (seq_len, d_model)
    pe = torch.zeros(seq_len, d_model, device=x.device)

    position = torch.arange(seq_len, device=x.device).unsqueeze(1)  # (seq_len, 1)
    div_term = torch.exp(torch.arange(0, d_model, 2, device=x.device) * (-math.log(10000.0) / d_model))  # (d_model/2)

    # 处理嵌入维度为奇数的情况
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term[:(d_model//2)])  # 防止越界

    # 添加维度以便广播： (1, seq_len, d_model)
    pe = pe.unsqueeze(0)

    # 加到输入上 (batch_size, seq_len, d_model)
    return x + pe
class PressureSkeletonDataset(Dataset):
    def __init__(self, pressure_data, skeleton_data, decoder_input):
        self.pressure_data = torch.FloatTensor(pressure_data)
        self.skeleton_data = torch.FloatTensor(skeleton_data)
        self.decoder_input = torch.FloatTensor(decoder_input)
    def __len__(self):
        return len(self.pressure_data)
    
    def __getitem__(self, idx):
        return self.pressure_data[idx], self.skeleton_data[idx],self.decoder_input[idx]    
class EnhancedSkeletonTransformer(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_encoder_layers, num_joints, num_dims=3, dropout=0.1,seq_length=3,window_size=5):
        super().__init__()

        # クラス属性としてnum_jointsを保存
        self.num_joints = num_joints
        self.num_dims = num_dims
        self.seq_length=seq_length
        self.window_size=window_size
        
        # 入力の特徴抽出を強化
        self.feature_extractor = nn.Sequential(
            nn.Linear(input_dim,d_model)
        )
        
        # より深いTransformerネットワーク
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model * 4,  
            dropout=dropout,
            batch_first=True,
            norm_first=True
        )
        
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_encoder_layers
        )

        self.decoder_feature_extractor = nn.Sequential(
            nn.Linear(num_joints*num_dims,d_model)
        )
        
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            batch_first=True,
            norm_first=True
        )
        
        self.transformer_decoder = nn.TransformerDecoder(
            decoder_layer,
            num_layers=num_encoder_layers
        )
        self.predict = nn.Sequential(
            nn.Linear(d_model*seq_length,d_model*window_size)
        )
        
        # 出力層の強化
        self.output_decoder = nn.Sequential(
            nn.Linear(d_model, d_model * 2),
            nn.LayerNorm(d_model * 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * 2, d_model),
            nn.ReLU(),
            nn.Linear(d_model, num_joints * num_dims)
        )
        # スケール係数（学習可能パラメータ）
        self.output_scale = nn.Parameter(torch.ones(1))

    
    def forward(self, x, decoder_input):
        batch_size=x.shape[0]
        
        # 特徴抽出
        features = self.feature_extractor(x)
        features = features.unsqueeze(1)
        decoder_input = self.decoder_feature_extractor(decoder_input)
        #decoder_input = add_positional_encoding(decoder_input)

        # Transformer処理
        transformer_output = self.transformer_encoder(features)
        transformer_output = self.transformer_decoder(decoder_input, transformer_output)
        #predict = transformer_output[:,transformer_output.shape[1]-1,:]

        predict=transformer_output.reshape(batch_size,-1)
        predict_next=self.predict(predict)
        predict_next=predict_next.reshape(batch_size,self.window_size,-1)
        
        # 出力生成とスケーリング
        output = self.output_decoder(predict_next)
        #output = self.constraint(output)
        output = output * self.output_scale  # 出力のスケーリング

        
        return output
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, save_path, device):
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        
        for pressure, skeleton, decoder_input in train_loader:
            # データをGPUに移動
            pressure = pressure.to(device)
            skeleton = skeleton.to(device)
            decoder_input = decoder_input.to(device)
            decoder_input = add_positional_encoding(decoder_input)
            
            pressure = add_gaussian_noise(pressure, noise_factor=0.1)
            decoder_input =  add_gaussian_noise(decoder_input, noise_factor=0.1)
            if torch.rand(1).item() < 0.95:
                decoder_input=torch.zeros_like(decoder_input)
            outputs = model(pressure,decoder_input)
            loss = criterion(outputs,skeleton)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for pressure, skeleton,decoder_input in val_loader:
                # データをGPUに移動
                pressure = pressure.to(device)
                skeleton = skeleton.to(device)
                decoder_input = decoder_input.to(device)
                
                outputs = model(pressure,decoder_input)
                loss = criterion(outputs, skeleton)
                val_loss += loss.item()
        
        # 平均損失の計算
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        
        # スケジューラのステップ
        scheduler.step(avg_val_loss)
        current_lr = optimizer.param_groups[0]['lr']
        
        print(f'Epoch {epoch+1}')
        print(f'Training Loss: {avg_train_loss:.4f}')
        print(f'Validation Loss: {avg_val_loss:.4f}')
        print(f'Learning Rate: {current_lr:.6f}')
        
        # モデルの保存
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_val_loss': best_val_loss,
            }
            torch.save(checkpoint, save_path)
            print(f'Model saved at epoch {epoch+1}')
        
        print('-' * 60)
class EnhancedSkeletonLoss_WithAngleConstrains(nn.Module):
    def __init__(self, alpha=1.0, gamma=0.5, window_size=5):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.window_size = window_size
        
    def forward(self, pred, target):
        # MSE損失
        # 假设 pred 和 target 的原始形状为 [batch_size, window_size * num_joints * 3]
        batch_size = int(pred.shape[0])
        num_joints = int(pred.shape[2] // 3)
        
        # 重塑为 [batch_size, window_size, num_joints, 3]
        pred_reshaped = pred.view(batch_size, self.window_size, num_joints, 3)
        target_reshaped = target.view(batch_size, self.window_size, num_joints, 3)
        
        # 定义关节点权重，默认全部为 1.0
        joint_weights = torch.tensor(0.2, device=pred.device)*torch.ones(num_joints, device=pred.device)
        # 对背骨的关节点（索引 0～4）赋予较高权重
        for idx in [0, 1, 2, 3, 4]:
            joint_weights[idx] = 2.0
        # 对两个腿的关节点（例如：一侧 13～16，另一侧 17～20）赋予较高权重
        for idx in [13, 14, 15, 16, 17, 18, 19, 20]:
            joint_weights[idx] = 2.0
        
        # 计算加权均方误差
        # 先计算每个坐标的平方误差，形状为 [batch_size, window_size, num_joints, 3]
        squared_diff = (pred_reshaped - target_reshaped) ** 2
        # 对坐标求和得到每个关节的误差，形状为 [batch_size, window_size, num_joints]
        squared_diff = squared_diff.sum(dim=-1)
        # 将关节点的权重扩展到 [1, 1, num_joints] 后相乘
        weighted_squared_diff = squared_diff * joint_weights.view(1, 1, num_joints)
        # 平均得到加权的均方误差
        mse_loss = weighted_squared_diff.mean()

        
        eps = 1e-6
        angle_loss = 0.0
        angle_pairs = [
            ((0, 1), (1, 2)),
            ((1, 2), (2, 3)),
            ((2, 3), (3, 4)),
            ((13, 17), (17, 18)),
            ((13, 17), (13, 14)),
            ((17, 18), (18, 19)),
            ((18, 19), (19, 20)),
            ((13, 14), (14, 15)),
            ((14, 15), (15, 16))
        ]
        for (bone1, bone2) in angle_pairs:
            # 预测向量计算
            pred_vec1 = pred_reshaped[:, :, bone1[1], :] - pred_reshaped[:, :, bone1[0], :]
            pred_vec2 = pred_reshaped[:, :, bone2[1], :] - pred_reshaped[:, :, bone2[0], :]
            dot_pred = (pred_vec1 * pred_vec2).sum(dim=-1)
            norm_pred1 = torch.norm(pred_vec1, dim=-1)
            norm_pred2 = torch.norm(pred_vec2, dim=-1)
            cos_pred = dot_pred / (norm_pred1 * norm_pred2 + eps)
            cos_pred = torch.clamp(cos_pred, -1.0, 1.0)
            
            # 目标向量计算
            target_vec1 = target_reshaped[:, :, bone1[1], :] - target_reshaped[:, :, bone1[0], :]
            target_vec2 = target_reshaped[:, :, bone2[1], :] - target_reshaped[:, :, bone2[0], :]
            dot_target = (target_vec1 * target_vec2).sum(dim=-1)
            norm_target1 = torch.norm(target_vec1, dim=-1)
            norm_target2 = torch.norm(target_vec2, dim=-1)
            cos_target = dot_target / (norm_target1 * norm_target2 + eps)
            cos_target = torch.clamp(cos_target, -1.0, 1.0)
            
            # 直接比较余弦值的差异
            angle_loss += F.mse_loss(cos_pred, cos_target)
        angle_loss = angle_loss / len(angle_pairs)
    
        return self.alpha * mse_loss + self.gamma*angle_loss

def load_model(model, optimizer, scheduler, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    epoch = checkpoint['epoch']
    best_val_loss = checkpoint['best_val_loss']

    return model, optimizer, scheduler, epoch, best_val_loss

In [3]:
def load_and_combine_data(file_pairs,seq_length,window_size):
    """複数のデータセットを読み込んで結合する"""
    all_skeleton_data = []
    all_pressure_left = []
    all_pressure_right = []
    all_decoder_input = []

    all_skeleton_label = []
    for skeleton_file, left_file, right_file in file_pairs:
        skeleton = pd.read_csv(skeleton_file)
        left = pd.read_csv(left_file, dtype=float, low_memory=False)
        right = pd.read_csv(right_file, dtype=float, low_memory=False)
        # データ長を揃える
        min_length = min(len(skeleton), len(left), len(right))

        num_joints_points=skeleton.shape[1]
        decoder_input = np.zeros((min_length,seq_length,num_joints_points))
        for i in range(min_length):
            for j in range(1,seq_length+1):
                if i-j<0:
                    continue
                else:
                    decoder_input[i,seq_length-j]=skeleton.iloc[i-j]

        skeleton_label = np.zeros((min_length,window_size,num_joints_points))
        for i in range(min_length):
            for j in range(window_size):
                if i+j<min_length:
                    skeleton_label[i,j]=skeleton.iloc[i+j]
        
        all_skeleton_data.append(skeleton.iloc[:min_length])
        all_pressure_left.append(left.iloc[:min_length])
        all_pressure_right.append(right.iloc[:min_length])
        all_decoder_input.append(decoder_input)
        all_skeleton_label.append(skeleton_label)
        
    return (pd.concat(all_skeleton_data, ignore_index=True),
            pd.concat(all_pressure_left, ignore_index=True),
            pd.concat(all_pressure_right, ignore_index=True),
            np.concatenate(all_decoder_input),
            np.concatenate(all_skeleton_label))

def preprocess_pressure_data(left_data, right_data):
    """圧力、回転、加速度データの前処理"""
    
    # 左足データから各種センサー値を抽出
    left_pressure = left_data.iloc[:, :35]  # 圧力センサーの列を適切に指定
    left_rotation = left_data.iloc[:, 35:38]  # 回転データの列を適切に指定
    left_accel = left_data.iloc[:, 38:41]  # 加速度データの列を適切に指定

    # 右足データから各種センサー値を抽出
    right_pressure = right_data.iloc[:, :35]  # 圧力センサーの列を適切に指定
    right_rotation = right_data.iloc[:, 35:38]  # 回転データの列を適切に指定
    right_accel = right_data.iloc[:, 38:41]  # 加速度データの列を適切に指定

    # データの結合(按列（属性）相拼接)
    pressure_combined = pd.concat([left_pressure, right_pressure], axis=1)
    rotation_combined = pd.concat([left_rotation, right_rotation], axis=1)
    accel_combined = pd.concat([left_accel, right_accel], axis=1)

    # NaN値を補正
    pressure_combined = pressure_combined.fillna(0.0)
    rotation_combined = rotation_combined.fillna(0.0)
    accel_combined = accel_combined.fillna(0.0)

    print("Checking pressure data for NaN or Inf...")
    print("Pressure NaN count:", pressure_combined.isna().sum().sum())
    print("Pressure Inf count:", np.isinf(pressure_combined).sum().sum())

    # 移動平均フィルタの適用
    window_size = 3
    pressure_combined = pressure_combined.rolling(window=window_size, center=True).mean()
    rotation_combined = rotation_combined.rolling(window=window_size, center=True).mean()
    accel_combined = accel_combined.rolling(window=window_size, center=True).mean()
    
    # NaN値を前後の値で補間
    pressure_combined = pressure_combined.bfill().ffill()
    rotation_combined = rotation_combined.bfill().ffill()
    accel_combined = accel_combined.bfill().ffill()

    # 正規化と標準化のスケーラー初期化
    pressure_normalizer = MinMaxScaler()
    rotation_normalizer = MinMaxScaler()
    accel_normalizer = MinMaxScaler()

    pressure_standardizer = StandardScaler(with_mean=True, with_std=True)
    rotation_standardizer = StandardScaler(with_mean=True, with_std=True)
    accel_standardizer = StandardScaler(with_mean=True, with_std=True)

    # データの正規化と標準化
    pressure_processed = pressure_standardizer.fit_transform(
        pressure_normalizer.fit_transform(pressure_combined)
    )
    rotation_processed = rotation_standardizer.fit_transform(
        rotation_normalizer.fit_transform(rotation_combined)
    )
    accel_processed = accel_standardizer.fit_transform(
        accel_normalizer.fit_transform(accel_combined)
    )
    
    # すべての特徴量を結合
    input_features = np.concatenate([
        pressure_processed,
        rotation_processed,
        accel_processed,
    ], axis=1)

    return input_features, {
        'pressure': {
            'normalizer': pressure_normalizer,
            'standardizer': pressure_standardizer
        },
        'rotation': {
            'normalizer': rotation_normalizer,
            'standardizer': rotation_standardizer
        },
        'accel': {
            'normalizer': accel_normalizer,
            'standardizer': accel_standardizer
        }
    }

In [4]:
def main():
    # データの読み込み
    data_pairs = [
        #
        # 第三回収集データ
        #
        # # 立ちっぱなし
         ('./data/20241115test3/Opti-track/Take 2024-11-15 03.20.00 PM.csv',
          './data/20241115test3/insoleSensor/20241115_152500_left.csv',
          './data/20241115test3/insoleSensor/20241115_152500_right.csv'),
        # お辞儀
        ('./data/20241115test3/Opti-track/Take 2024-11-15 03.26.00 PM.csv',
         './data/20241115test3/insoleSensor/20241115_153100_left.csv', 
         './data/20241115test3/insoleSensor/20241115_153100_right.csv'),
        # 体の横の傾け
        ('./data/20241115test3/Opti-track/Take 2024-11-15 03.32.00 PM.csv', 
         './data/20241115test3/insoleSensor/20241115_153700_left.csv', 
        './data/20241115test3/insoleSensor/20241115_153700_right.csv'),
        # 立つ座る
        ('./data/20241115test3/Opti-track/Take 2024-11-15 03.38.00 PM.csv', 
         './data/20241115test3/insoleSensor/20241115_154300_left.csv', 
         './data/20241115test3/insoleSensor/20241115_154300_right.csv'),
        # スクワット
        ('./data/20241115test3/Opti-track/Take 2024-11-15 03.44.00 PM.csv', 
         './data/20241115test3/insoleSensor/20241115_154900_left.csv', 
         './data/20241115test3/insoleSensor/20241115_154900_right.csv'),
         # 総合(test3)
        #('./data/20241115test3/Opti-track/Take 2024-11-15 03.50.00 PM.csv', 
        # './data/20241115test3/insoleSensor/20241115_155500_left.csv', 
        # './data/20241115test3/insoleSensor/20241115_155500_right.csv'),

        # 釘宮くん
        ('./data/20241212test4/Opti-track/Take 2024-12-12 03.06.59 PM.csv',
         './data/20241212test4/insoleSensor/20241212_152700_left.csv', 
         './data/20241212test4/insoleSensor/20241212_152700_right.csv'),
        # 百田くん
        ('./data/20241212test4/Opti-track/Take 2024-12-12 03.45.00 PM.csv', 
         './data/20241212test4/insoleSensor/20241212_160501_left.csv', 
         './data/20241212test4/insoleSensor/20241212_160501_right.csv'),
        # # # # 渡辺(me)
         ('./data/20241212test4/Opti-track/Take 2024-12-12 04.28.00 PM.csv', 
          './data/20241212test4/insoleSensor/20241212_164800_left.csv', 
          './data/20241212test4/insoleSensor/20241212_164800_right.csv'),
        # にるぱむさん
        ('./data/20241212test4/Opti-track/Take 2024-12-12 05.17.59 PM.csv', 
         './data/20241212test4/insoleSensor/20241212_173800_left.csv', 
         './data/20241212test4/insoleSensor/20241212_173800_right.csv')
    ]
    
    # データの読み込みと結合
    seq_length=3
    window_size=1
    skeleton_data, pressure_data_left, pressure_data_right, decoder_input, skeleton_label = load_and_combine_data(data_pairs,seq_length,window_size)
    
    # numpy配列に変換
    skeleton_data = skeleton_data.to_numpy()
    decoder_input = decoder_input

    # 圧力、回転、加速度データの前処理
    input_features, sensor_scalers = preprocess_pressure_data(
        pressure_data_left,
        pressure_data_right
    )
    print(input_features.shape)
    
    # データの分割
    train_input, val_input, train_skeleton, val_skeleton, train_decoder_input, val_decoder_input = train_test_split(
        input_features, 
        skeleton_label,
        decoder_input,
        test_size=0.2, 
        random_state=42
    )
    print(train_decoder_input.shape)
    print(val_decoder_input.shape)
    # デバイスの設定
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # モデルのパラメータ設定
    input_dim = input_features.shape[1]  # 圧力+回転+加速度の合計次元数
    d_model = 512
    nhead = 8
    num_encoder_layers = 6
    num_joints = skeleton_data.shape[1] // 3  # 3D座標なので3で割る
    dropout = 0.1
    batch_size = 128

    # データローダーの設定
    train_dataset = PressureSkeletonDataset(train_input, train_skeleton, train_decoder_input)
    val_dataset = PressureSkeletonDataset(val_input, val_skeleton, val_decoder_input)

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    print("Checking final training and validation data...")
    print("Train input NaN count:", np.isnan(train_input).sum(), "Inf count:", np.isinf(train_input).sum())
    print("Train skeleton NaN count:", np.isnan(train_skeleton).sum(), "Inf count:", np.isinf(train_skeleton).sum())
    
    # モデルの初期化
    model = EnhancedSkeletonTransformer(
        input_dim= input_features.shape[1], # input_dim,
        d_model= d_model,
        nhead= nhead,
        num_encoder_layers= num_encoder_layers,
        num_joints=num_joints,
        num_dims=3,
        dropout=dropout,
        seq_length=seq_length,
        window_size=window_size
    ).to(device)

    # 損失関数、オプティマイザ、スケジューラの設定
    # criterion = torch.nn.MSELoss()  # 必要に応じてカスタム損失関数に変更可能
    criterion = EnhancedSkeletonLoss_WithAngleConstrains(alpha=1.0,gamma=0.5,window_size=window_size)
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=0.0001,
        weight_decay=0.001,
        betas=(0.9, 0.999)
    )
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=5
    )

    # トレーニング実行
    train_model(
        model,
        train_loader,
        val_loader,
        criterion,
        optimizer,
        scheduler,
        num_epochs=300,
        save_path='./weight/best_skeleton_model.pth',
        device=device,
    )

    # モデルの保存
    final_checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'sensor_scalers': sensor_scalers,
        # 'skeleton_skaler': skeleton_scaler,
        'model_config': {
            'input_dim': input_dim,
            'd_model': d_model,
            'nhead': nhead,
            'num_encoder_layers': num_encoder_layers,
            'num_joints': num_joints
        }
    }
    torch.save(final_checkpoint, './weight/final_skeleton_model.pth')
    
if __name__ == "__main__":
    main()

Checking pressure data for NaN or Inf...
Pressure NaN count: 0
Pressure Inf count: 0
(62782, 82)
(50225, 3, 63)
(12557, 3, 63)
Using device: cuda:0
Checking final training and validation data...
Train input NaN count: 0 Inf count: 0
Train skeleton NaN count: 0 Inf count: 0


/opt/conda/lib/python3.12/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


Epoch 1
Training Loss: 914465.2783
Validation Loss: 699580.5417
Learning Rate: 0.000100
Model saved at epoch 1
------------------------------------------------------------
Epoch 2
Training Loss: 450993.6691
Validation Loss: 218231.1545
Learning Rate: 0.000100
Model saved at epoch 2
------------------------------------------------------------
Epoch 3
Training Loss: 89195.3209
Validation Loss: 37378.4151
Learning Rate: 0.000100
Model saved at epoch 3
------------------------------------------------------------
Epoch 4
Training Loss: 33774.5938
Validation Loss: 32452.1544
Learning Rate: 0.000100
Model saved at epoch 4
------------------------------------------------------------
Epoch 5
Training Loss: 27401.0942
Validation Loss: 24911.6005
Learning Rate: 0.000100
Model saved at epoch 5
------------------------------------------------------------
Epoch 6
Training Loss: 25881.7049
Validation Loss: 23988.9164
Learning Rate: 0.000100
Model saved at epoch 6
-------------------------------------

In [5]:
def compute_exponential_weights(k, m):
    """计算指数衰减权重 w_i = exp(-m * i)，并归一化"""
    indices = torch.arange(k)  # 生成 i = 0, 1, ..., k-1
    weights = torch.exp(-m * indices)  # 计算 w_i
    return weights / weights.sum()  # 归一化，使得所有权重之和为 1

# 示例：计算最近 5 个时间步的权重
k = 10  # 观察的时间窗口
m = 0.5  # 衰减因子（越大表示衰减越快）

weights = compute_exponential_weights(k, m)
print(weights)

tensor([0.3961, 0.2403, 0.1457, 0.0884, 0.0536, 0.0325, 0.0197, 0.0120, 0.0073,
        0.0044])


In [6]:
class EnhancedSkeletonLoss(nn.Module):
    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = alpha
        
    def forward(self, pred, target):
        # MSE損失
        mse_loss = F.mse_loss(pred, target)
        batch_size = int(pred.shape[0])
        # 変化量の損失
        motion_loss = F.mse_loss(
            pred[1:] - pred[:-1],
            target[1:] - target[:-1]
        )
        return self.alpha * mse_loss/batch_size

In [7]:
def preprocess_pressure_data(left_data, right_data):
    """圧力、回転、加速度データの前処理"""
    # 左足データから各種センサー値を抽出
    left_pressure = left_data.iloc[:, :35]
    left_rotation = left_data.iloc[:, 35:38]
    left_accel = left_data.iloc[:, 38:41]

    # 右足データから各種センサー値を抽出
    right_pressure = right_data.iloc[:, :35]
    right_rotation = right_data.iloc[:, 35:38]
    right_accel = right_data.iloc[:, 38:41]

    # データの結合
    pressure_combined = pd.concat([left_pressure, right_pressure], axis=1)
    rotation_combined = pd.concat([left_rotation, right_rotation], axis=1)
    accel_combined = pd.concat([left_accel, right_accel], axis=1)

    # NaN値を補正
    pressure_combined = pressure_combined.ffill().bfill()
    rotation_combined = rotation_combined.ffill().bfill()
    accel_combined = accel_combined.ffill().bfill()

    # 移動平均フィルタの適用
    window_size = 3
    pressure_combined = pressure_combined.rolling(window=window_size, center=True).mean()
    rotation_combined = rotation_combined.rolling(window=window_size, center=True).mean()
    accel_combined = accel_combined.rolling(window=window_size, center=True).mean()
    
    # NaN値を補間
    pressure_combined = pressure_combined.ffill().bfill()
    rotation_combined = rotation_combined.ffill().bfill()
    accel_combined = accel_combined.ffill().bfill()

    # 正規化と標準化
    pressure_normalizer = MinMaxScaler()
    rotation_normalizer = MinMaxScaler()
    accel_normalizer = MinMaxScaler()

    pressure_standardizer = StandardScaler(with_mean=True, with_std=True)
    rotation_standardizer = StandardScaler(with_mean=True, with_std=True)
    accel_standardizer = StandardScaler(with_mean=True, with_std=True)

    # データの正規化と標準化
    pressure_processed = pressure_standardizer.fit_transform(
        pressure_normalizer.fit_transform(pressure_combined)
    )
    rotation_processed = rotation_standardizer.fit_transform(
        rotation_normalizer.fit_transform(rotation_combined)
    )
    accel_processed = accel_standardizer.fit_transform(
        accel_normalizer.fit_transform(accel_combined)
    )

    # 1次微分と2次微分の計算
    pressure_grad1 = np.gradient(pressure_processed, axis=0)
    pressure_grad2 = np.gradient(pressure_grad1, axis=0)
    
    rotation_grad1 = np.gradient(rotation_processed, axis=0)
    rotation_grad2 = np.gradient(rotation_grad1, axis=0)
    
    accel_grad1 = np.gradient(accel_processed, axis=0)
    accel_grad2 = np.gradient(accel_grad1, axis=0)

    # すべての特徴量を結合（246次元になるはず）
    input_features = np.concatenate([
        pressure_processed,  # 原特徴量
        #pressure_grad1,     # 1次微分
        #pressure_grad2,     # 2次微分
        rotation_processed,
        #rotation_grad1,
        #rotation_grad2,
        accel_processed,
        #accel_grad1,
        #accel_grad2
    ], axis=1)

    return input_features

def load_and_preprocess_data(file_pairs):
    predictions_all = []
    
    for skeleton_file, left_file, right_file in file_pairs:
        skeleton_data = pd.read_csv(skeleton_file)
        pressure_data_left = pd.read_csv(left_file)
        pressure_data_right = pd.read_csv(right_file)
        
        input_features = preprocess_pressure_data(pressure_data_left, pressure_data_right)
        min_length = min(len(skeleton_data), len(input_features))
        
        input_features = input_features.iloc[:min_length]
        skeleton_data = skeleton_data.iloc[:min_length]
        
        predictions_all.append((input_features, skeleton_data))
    
    return predictions_all

def predict_skeleton():
    try:
        # データの読み込みと前処理
        skeleton_data = pd.read_csv('./data/20241115test3/Opti-track/Take 2024-11-15 03.50.00 PM.csv')
        pressure_data_left = pd.read_csv('./data/20241115test3/insoleSensor/20241115_155500_left.csv', skiprows=1)
        pressure_data_right = pd.read_csv('./data/20241115test3/insoleSensor/20241115_155500_right.csv', skiprows=1)

        # 入力データの前処理
        input_features = preprocess_pressure_data(pressure_data_left, pressure_data_right)
        min_length=min(input_features.shape[0],skeleton_data.shape[0])
 
        # 入力の次元数を取得
        seq_length = 3
        window_size = 1
        m=1
        input_dim = input_features.shape[1]
        num_joints = skeleton_data.shape[1] // 3

        # デバイスの設定
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")
        skeleton_data=torch.FloatTensor(np.array(skeleton_data)[:min_length]).to(device)

        # モデルの初期化（固定パラメータを使用）
        model = EnhancedSkeletonTransformer(
            input_dim=input_dim,
            d_model=512,
            nhead=8,
            num_encoder_layers=6,
            num_joints=num_joints,
            num_dims=3,
            dropout=0.1,
            seq_length=seq_length,
            window_size=window_size
        ).to(device)

        # チェックポイントの読み込み（weights_only=Trueを追加）
        checkpoint = torch.load('./weight/best_skeleton_model.pth', map_location=device, weights_only=True)
        
        # モデルの重みを読み込み
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        print("Model loaded successfully")

        action=torch.zeros((min_length+10,window_size,63)).to(device)
        num=np.zeros(min_length+10)
        print(action.shape)
        # 予測の実行
        print("Making predictions...")
        predictions=torch.zeros(min_length,63).to(device)
        with torch.no_grad():
            skeleton_last=torch.zeros((seq_length,63))
            skeleton_last=skeleton_last.unsqueeze(0).to(device)
            for i in range(min_length):
                input_tensor = torch.FloatTensor(input_features)[i].to(device)
                input_tensor = input_tensor.unsqueeze(0).to(device)
                
                skeleton_last_pos=add_positional_encoding(skeleton_last)
                skeleton_predict_seq=model(input_tensor,skeleton_last)
                skeleton_predict_seq=skeleton_predict_seq.squeeze(0)
                skeleton_predict=torch.zeros(63).to(device)
                for j in range(window_size):
                    action[i+j,int(num[i+j])]=skeleton_predict_seq[j,:]
                    num[i+j]+=1
                    #print(f"j={j},i+j={i+j},num[i+j}]={int(num[i+j])}")
                weights = compute_exponential_weights(int(num[i]), m).to(device)
                for j in range(int(num[i])):
                    skeleton_predict+=weights[j]*action[i,int(num[i])-1-j]
                predictions[i]=skeleton_predict
                for j in range(seq_length-1):
                    skeleton_last[0,j]=skeleton_last[0,j+1]
                skeleton_last[0,seq_length-1]=skeleton_predict
        '''
        # 予測の実行
        print("Making predictions...")
        predictions=torch.zeros(min_length,63).to(device)
        with torch.no_grad():
            input_tensor = torch.FloatTensor(input_features).to(device)
            input_tensor=input_tensor.to(device)
            print(input_tensor.shape,skeleton_data.shape)
            predictions=model(input_tensor,skeleton_data)
        '''
        print(f"Prediction shape: {predictions.shape}")
        criterion = EnhancedSkeletonLoss(alpha=1.0,)
        criterion1 = EnhancedSkeletonLoss_WithAngleConstrains(alpha=1.0,gamma=0.1,window_size=1)
        predictionsa = predictions.unsqueeze(1)
        skeleton_dataa = skeleton_data.unsqueeze(1)
        loss=criterion(predictionsa,skeleton_dataa)
        print(f"Loss: {loss}")
        loss1=criterion1(predictionsa,skeleton_dataa)
        print(f"Loss: {loss1}")
        predictions = predictions.cpu().numpy()
        return predictions

    except Exception as e:
        print(f"Error during prediction: {str(e)}")
        raise

def save_predictions(predictions, output_file='./output/predicted_skeleton.csv'):
    try:
        # 予測結果をデータフレームに変換
        num_joints = predictions.shape[1] // 3
        columns = []
        for i in range(num_joints):
            columns.extend([f'X.{i*2+1}', f'Y.{i*2+1}', f'Z.{i*2+1}'])
        
        df_predictions = pd.DataFrame(predictions, columns=columns)
        df_predictions.to_csv(output_file, index=False)
        print(f"Predictions saved to {output_file}")
        
    except Exception as e:
        print(f"Error saving predictions: {str(e)}")
        raise

def main():
    try:
        print("Starting prediction process...")
        predictions = predict_skeleton()
        
        print("\nSaving predictions...")
        save_predictions(predictions)
        print(predictions)
        
        print("Prediction process completed successfully!")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Starting prediction process...
Using device: cuda


/opt/conda/lib/python3.12/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


Model loaded successfully
torch.Size([3004, 1, 63])
Making predictions...
Prediction shape: torch.Size([2994, 63])
Loss: 1.9015629291534424
Loss: 23642.900390625

Saving predictions...
Predictions saved to ./output/predicted_skeleton.csv
[[ 5.95058870e+00  8.80480286e+02  1.30444992e+00 ... -8.03655624e+01
   8.88666306e+01 -1.08369812e+02]
 [ 5.28601599e+00  8.74848572e+02  1.16547906e+00 ... -7.58727646e+01
   7.67515945e+01 -1.04952759e+02]
 [ 4.04099131e+00  8.80096069e+02  1.04391479e+00 ... -7.08359909e+01
   8.52122803e+01 -1.06465385e+02]
 ...
 [ 6.22371960e+00  8.81900146e+02 -6.85157835e-01 ... -7.94677658e+01
   9.05017471e+01 -1.47679459e+02]
 [ 6.23053646e+00  8.81967041e+02 -4.05712605e-01 ... -8.01860428e+01
   9.03172913e+01 -1.47116211e+02]
 [ 6.22929716e+00  8.81923401e+02 -1.83740184e-01 ... -8.06514130e+01
   9.01947632e+01 -1.46984451e+02]]
Prediction process completed successfully!
